# Week 6 Day 3: Fitting

## Objectives

* Learn how to perform a simple fit on data
* Learn about various tools for data fitting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.linspace(0, 200, 400)
x_data = np.arange(0, 225, 25)
x_data

In [ ]:
y_data = np.array([10.6, 16.0, 45.0, 83.5, 52.8,
                   19.9, 10.8, 8.25, 4.7])
e_data = np.array([9.34, 17.9, 41.5, 85.5, 51.5,
                   21.5, 10.8, 6.29, 4.14])
e_data = np.array([5, 7, 10, 12,
                   10, 7, 5, 4, 4])

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x_data, y_data, e_data, fmt='.')
plt.show()

In [ ]:
def interp_langrange(x_data, y_data, x):
    'A custom lagrange fit function.'
    
    # The shape will be N x N x n, where N is the number of points
    # in x and y, and n is the number of points in the result.
    x = np.asarray(x)
    x = x.reshape(1,1,-1)
    
    # Make grid. Have one "throw away" diminsion to make [N,N,1] shaped arrays
    x_n, x_i, _ = np.meshgrid(x_data, x_data, np.array([1]))
    
    # Using a mask here avoids numpy warnings and makes the product easy
    x_n = np.ma.array(x_n, mask=np.eye(x_data.shape[0]))
    x_i = np.ma.array(x_i, mask=np.eye(x_data.shape[0]))
    
    V = (x - x_n) / (x_i - x_n)
    v = np.prod(V, axis=1).data # Convert back to non-masked arrays
    
    return y_data @ v

# This was added to test interp_langrange
x_d2 = np.array([0,1,2,4])
y_d2 = np.array([-12, -12, -24, -60])

interp_langrange(x_d2, y_d2, [.5, 4]) # should be -10.125 and -60

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, interp_langrange(x_data, y_data, x))
ax.plot(x_data, y_data, 'o')
plt.show()


In [ ]:
from scipy import interpolate

In [ ]:
p = interpolate.lagrange(x_data, y_data)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, p(x))
ax.plot(x_data, y_data, 'o')
plt.show()

In [ ]:
spline = interpolate.CubicSpline(x_data, y_data)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, spline(x))
ax.plot(x_data, y_data, 'o')
plt.show()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def f(E, f_r, E_r, Γ):
    return f_r / ((E - E_r)**2 + Γ**2/4)

In [ ]:
popt, pcov = curve_fit(f, x_data, y_data,
                       p0=[1.,1.,1.],
                       sigma=e_data,
                       absolute_sigma=True)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x_data, y_data, e_data, fmt='.')
ax.plot(x, f(x, *popt))
plt.show()

## Advanced usage: The Uncertainties package

Let's use that correlation matrix that `curve_fit` returned!

In [ ]:
from uncertainties import correlated_values, unumpy

In [ ]:
copt = correlated_values(popt, pcov)

In [ ]:
def plot_with_unc(x, y, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
        
    y_nv = unumpy.nominal_values(y)
    y_sd = unumpy.std_devs(y)
    
    line, = ax.plot(x, y_nv)
    ax.fill_between(x, y_nv - y_sd, y_nv + y_sd, color=line.get_color(), alpha=.2)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x_data, y_data, e_data, fmt='.')
plot_with_unc(x, f(x, *copt), ax=ax)
plt.show()

## Linear least squares

In [ ]:
t = np.linspace(1,2)
x = np.array([1., 1.1, 1.24, 1.35, 1.451, 1.5, 1.92])
y = np.array([0.52, 0.8, 0.7, 1.8, 2.9, 2.9, 3.6])
sig = np.array([.1, .1, .2, .3, .2, .1, .1])

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x, y, sig, fmt='.')
plt.show()

## Manual least squares fitting:

Fitting function:

$$
g(x_i) = a_0 x_i^2 + a_1 x_i + a_2
\tag{1}
$$

If you compute the minimum of $\chi^2$, you get:

$$
\sum _i
\frac{g(x_i)}{\sigma_i^2}
\frac{\partial g ( x_i)}{\partial a_m}
=
\sum _i
\frac{y_i}{\sigma_i^2}
\frac{\partial g ( x_i)}{\partial a_m}
\tag{2}
$$

We can rewrite this as a matrix expression, expanding (2) in terms of (1):

$$
A x = b
\tag{3}
$$

$$
A = \left[
\begin{matrix}
 \sum_i \frac{x_i^4}{\sigma_i^2} & \sum_i \frac{x_i^3}{\sigma_i^2} & \sum_i \frac{x_i^2}{\sigma_i^2} \\
 \sum_i \frac{x_i^3}{\sigma_i^2} & \sum_i \frac{x_i^2}{\sigma_i^2} & \sum_i \frac{x_i^1}{\sigma_i^2} \\
 \sum_i \frac{x_i^2}{\sigma_i^2} & \sum_i \frac{x_i  }{\sigma_i^2} & \sum_i \frac{1    }{\sigma_i^2} \\
\end{matrix}
\right]
\tag{3.1}
$$

$$
b = \left[
\begin{matrix}
 \sum_i \frac{y_i x_i^2}{\sigma_i^2} \\
 \sum_i \frac{y_i x_i  }{\sigma_i^2} \\
 \sum_i \frac{y_i      }{\sigma_i^2} \\
\end{matrix}
\right]
\tag{3.2}
$$

$$
x = \left[
\begin{matrix}
 a_2 \\
 a_1 \\
 a_0 \\
\end{matrix}
\right]
\tag{3.3}
$$

In [ ]:
sig2 = sig**2
ss = np.array([
    np.sum(x**4/sig2), np.sum(x**3/sig2), np.sum(x**2/sig2), np.sum(x/sig2), np.sum(1/sig2)
])
A = np.stack([ss[:3], ss[1:4], ss[2:]])

b = np.array([np.sum(x**2*y/sig2), np.sum(x*y/sig2), np.sum(y/sig2)])

We could have been much more clever. Looking at the above expression for M, we could instead have created the powers, then raise to a matrix of powers! 

In [ ]:
power = sum(np.mgrid[2:-1:-1,2:-1:-1])
Ap = np.sum(x[:,None,None]**power / sig[:,None,None]**2, axis=0)
assert np.all(A == Ap)

Exercise for the reader: try the same trick on `b`.

In [ ]:
xvec = np.linalg.solve(A, b)

In [ ]:
p = np.poly1d(xvec)
fig, ax = plt.subplots()
ax.errorbar(x, y, sig, fmt='.')
ax.plot(t, p(t))
plt.show()

Or, we could have just done this with the polyfit function in numpy:

In [ ]:
xvec = np.polyfit(x, y, 2, w=1/sig)
p = np.poly1d(xvec)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x, y, sig, fmt='.')
ax.plot(t, p(t))
plt.show()

Or the more general scipy `curve_fit`:

In [ ]:
def new_p(x, *values):
    return np.poly1d(values)(x)

xvec, _ = curve_fit(new_p, x, y, p0=[1,1,1],sigma=sig)
p = np.poly1d(xvec)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x, y, sig, fmt='.')
ax.plot(t, p(t))
plt.show()